In [42]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import pi
from PIL import Image
from pathlib import Path
import warnings

plt.rcParams['axes.axisbelow'] = True

In [43]:
class WaveEq:
    def __init__(self, T = 3., r_min = 0.0, r_max = 1.8, c = 1.5, a = 0.6, b = 1.2, I = 100, CFL = 0.5, d = 3, img_path = 'default', gif_path = 'default'):
        self.d = d
        self.I = I
        self.r_min = r_min
        self.r_max = r_max
        self.c = c
        self.a = a
        self.b = b
        self.i = np.arange(self.I + 2)
        self.h = (self.r_max - self.r_min) / I
        self.r_i = np.array([self.r_min + (iter - 0.5) * self.h for iter in self.i])
        self.T = T
        self.CFL = CFL
        self.tau = self.CFL * self.h / self.c
        self.t_i = np.linspace(0, self.T, int(self.T / self.tau + 1))
        self.f_i = np.zeros(self.I + 2)

        self.curr_time_step = 0
        self.curr_time = 0
        self.task_completed = False

        self._images_path = Path('./img/' + img_path)
        self._gifs_path = Path('./gifs/' + gif_path)
        if not self._images_path.is_dir():
            self._images_path.mkdir(parents=True, exist_ok=True)
        if not self._gifs_path.is_dir():
            self._gifs_path.mkdir(parents=True, exist_ok=True)


    def solutionStep(self, save = False):
        if(self.curr_time_step == 0):
            self.u_p = self.v0(self.r_i)
            self.dv0_i = self.dv0(self.r_i)
            self.du_t = lambda x: 0.0
            self.du_t_i = self.du_t(self.r_i)
            self.v0_i = self.v0(self.r_i)
            self.g_i = self.g(self.r_i)
            self.s_i = self.s(self.r_i)

            self.d2u_t_i = self.c ** 2 / self.r_i ** (self.d - 1) * ((self.d - 1) * self.r_i ** (self.d - 2) * self.v0_i * self.g_i \
                                                                     + self.r_i ** (self.d - 1) * (self.g_i ** 2 * self.v0_i + self.v0_i * self.s_i))
            
            self.u = self.u_p + self.tau * self.du_t_i + self.tau ** 2 * self.d2u_t_i * 0.5
            self.u_n = np.zeros_like(self.u)
            
            self.curr_time += self.tau
            self.curr_time_step += 1

            self.curr_time += self.tau
            self.curr_time_step += 1

        elif self.curr_time <= self.T:
            for i in range(1, self.I + 1):
                self.u_n[i] = 2 * self.u[i] - self.u_p[i] + (self.tau * self.c / self.h) ** 2 / self.r_i[i] ** (self.d - 1) * ((self.r_i[i] + 0.5 * self.h) ** (self.d - 1) * (self.u[i + 1] - self.u[i]) \
                                                                                                                                - (self.r_i[i] - 0.5 * self.h) ** (self.d - 1) * (self.u[i] - self.u[i - 1])) + self.tau ** 2 * self.f_i[i]

            self.u_n[0] = self.u_n[1]
            self.u_n[-1] = self.u_n[-2]

            if save:
                self.saveSnap()

            self.u_p = self.u.copy()
            self.u = self.u_n.copy()
            self.curr_time_step += 1
            self.curr_time += self.tau

        else:
            self.task_completed = True

    def v0(self, r):
        return np.piecewise(r, 
                            [r <= self.a, (r > self.a) & (r < self.b), r >= self.b], 
                            [lambda r: 0.,  
                            lambda r: np.exp((-4 * np.power((2 * r - self.a - self.b), 2)) / (np.power((self.b - self.a), 2) - np.power((2 * r - self.a - self.b), 2))),
                            lambda r: 0.])

    
    def g(self, r):
        return np.piecewise(r, 
                            [r <= self.a, (r > self.a) & (r < self.b), r >= self.b], 
                            [lambda r: 0.,  
                            lambda r: -(2 * r - self.a - self.b) * (self.a - self.b) ** 2 / (((self.b - r) * (self.a - r)) ** 2),
                            lambda r: 0.])
    
    def s(self, r):
        return np.piecewise(r, 
                            [r <= self.a, (r > self.a) & (r < self.b), r >= self.b], 
                            [lambda r: 0.,  
                            lambda r: 2 * (self.a - self.b) ** 2 * (self.a ** 2 + self.a * (self.b - 3 * r) + self.b ** 2 - 3 * self.b * r + 3 * r ** 2) / ((self.a - r) ** 3 * (self.b - r) ** 3),
                            lambda r: 0.])
    
    def dv0(self, r):
        return np.piecewise(r, 
                            [r <= self.a, (r > self.a) & (r < self.b), r >= self.b],
                            [lambda r: 0.,
                            lambda r: self.g(r) * self.v0(r),
                            lambda r: 0.])
    
    def createGif(self):
        p = self._images_path.as_posix()
        file_count = sum(1 for file in self._images_path.iterdir() if file.is_file())
        image_path_list = [p + f'/img_{i}.png' for i in range(file_count)]
        image_list = [Image.open(file) for file in image_path_list]
        image_list[0].save(
                self._gifs_path.joinpath(f'waveEquation{self.d}D-h{self.h}-CFL{self.CFL}.gif'),
                save_all=True,
                append_images=image_list[1:],
                duration=self.T / len(self.t_i),
                loop=0)
        
    def saveSnap(self):
        plt.figure(figsize=(10, 6))
        plt.scatter(self.r_i, self.u_n, color = 'black', s = 10)
        plt.plot(self.r_i, self.u_n, color = 'black')
        plt.ylim(self._ylim)
        plt.title(f'Решение волнового уравнения\nt = {self.t_i[self.curr_time_step]:.4f}, CFL = {self.CFL}, h = {self.h}, {self.d}D', fontsize=14)
        plt.grid(which='major', linestyle='-')
        plt.grid(which='minor', linestyle='--')
        plt.minorticks_on()
        plt.savefig(self._images_path.joinpath(f'img_{self.curr_time_step}.png'), 
                transparent = False,  
                facecolor = 'white'
                )
        plt.close()

    def printParams(self):
        print('d\t', self.d)
        print('I\t', self.I)
        print('r_min\t', self.r_min)
        print('r_max\t', self.r_max)
        print('c\t', self.c)
        print('a\t', self.a)
        print('b\t', self.b)
        print('i\t', self.i)
        print('h\t', self.h)
        print('r_i\t', self.r_i)
        print('T\t', self.T)
        print('CFL\t', self.CFL)
        print('tau\t', self.tau)
        print('t_i\t', self.t_i)

In [44]:
# wave1d = WaveEq(I = 180, d = 1, img_path='1d', gif_path='1d')
# while(wave1d.curr_time <= wave1d.T):
#     wave1d.solutionStep()

# wave2d = WaveEq(I = 180, d = 2, img_path='2d', gif_path='2d')
# while(wave2d.curr_time <= wave2d.T):
#     wave2d.solutionStep()

# wave3d = WaveEq(I = 180, d = 3, img_path='3d', gif_path='3d')
# while(wave3d.curr_time <= wave3d.T):
#     wave3d.solutionStep()

# wave1d.createGif()
# wave2d.createGif()
# wave3d.createGif()

In [54]:
alpha = 3
base_I = 200
T = 0.01
d = 1
CFL = 0.001

wave1d_h   = WaveEq(c = 1.5,  T = T, I = base_I, d = d, CFL = CFL, img_path= 'test_h')
wave1d_ah  = WaveEq(c = 1.5,  T = T, I = base_I * alpha, d = d, CFL = CFL, img_path= 'test_ah')
wave1d_a2h = WaveEq(c = 1.5,  T = T, I = base_I * (alpha ** 2), d = d, CFL = CFL, img_path= 'test_a2h')

norm1d_chisl_C = 0
norm1d_znam_C = 0

norm1d_chisl_L2 = 0
norm1d_znam_L2 = 0

beta1d_C = np.zeros(len(wave1d_h.t_i))
beta1d_L2 = np.zeros(len(wave1d_h.t_i))

wave1d_h.solutionStep()
wave1d_ah.solutionStep()
wave1d_a2h.solutionStep()


# print(wave1d_h.r_i[0:-1])
# print(wave1d_ah.r_i[0 : -(alpha - 1):alpha])
# print(wave1d_a2h.r_i[0: -(alpha**2 - alpha - 1):alpha**2])


diffs1d_chisl = wave1d_h.u_p[1:-1] - wave1d_ah.u_p[alpha - 1 : -(alpha - 1) : alpha]
diffs1d_znam = wave1d_ah.u_p[alpha - 1 : -(alpha - 1) : alpha] - wave1d_a2h.u_p[alpha**2 - alpha - 1 : -(alpha**2 - alpha - 1) : alpha ** 2]

norm1d_chisl_C = np.linalg.norm(diffs1d_chisl, ord= np.inf)
norm1d_znam_C = np.linalg.norm(diffs1d_znam, ord= np.inf)
beta1d_C[0] = norm1d_chisl_C / norm1d_znam_C

norm1d_chisl_L2 = np.linalg.norm(diffs1d_chisl, ord= 2)
norm1d_znam_L2 = np.linalg.norm(diffs1d_znam, ord= 2)
beta1d_L2[0] = norm1d_chisl_L2 / norm1d_znam_L2

def snap(n):
    plt.figure(figsize=(10, 6))
    plt.plot(wave1d_h.r_i[1:-1], diffs1d_chisl, color = 'red')
    plt.plot(wave1d_h.r_i[1:-1], diffs1d_znam, color = 'blue')
    plt.ylim((-0.1, 0.1))
    plt.title(f'Решение волнового уравнения\nn = {0:.4f}, CFL = {CFL}, {d}D', fontsize=14)
    plt.grid(which='major', linestyle='-')
    plt.grid(which='minor', linestyle='--')
    plt.minorticks_on()

    plt.savefig(f'img_test/img_{n}.png', 
            transparent = False,  
            facecolor = 'white'
            )
    plt.close()

snap(0)

for i in range(2 * alpha - 2):
    wave1d_ah.solutionStep()

for i in range(2 * alpha ** 2 - 2):
    wave1d_a2h.solutionStep()

# n = 1
diffs1d_chisl = wave1d_h.u[1:-1] - wave1d_ah.u[alpha - 1 : -(alpha - 1) : alpha]
diffs1d_znam = wave1d_ah.u[alpha - 1 : -(alpha - 1) : alpha] - wave1d_a2h.u[alpha**2 - alpha - 1 : -(alpha**2 - alpha - 1) : alpha ** 2]

norm1d_chisl_C = np.linalg.norm(diffs1d_chisl, ord= np.inf)
norm1d_znam_C = np.linalg.norm(diffs1d_znam, ord= np.inf)
beta1d_C[1] = norm1d_chisl_C / norm1d_znam_C

norm1d_chisl_L2 = np.linalg.norm(diffs1d_chisl, ord= 2)
norm1d_znam_L2 = np.linalg.norm(diffs1d_znam, ord= 2)
beta1d_L2[1] = norm1d_chisl_L2 / norm1d_znam_L2

snap(1)

def timeStep(save = False):
    wave1d_h.solutionStep(save= save)
    for i in range(alpha):
        wave1d_ah.solutionStep(save= save)
    for i in range(alpha**2):   
        wave1d_a2h.solutionStep(save= save)


# n > 1
n = 2
while(n < wave1d_h.t_i.size):
    timeStep()
    diffs1d_chisl = wave1d_h.u[1:-1] - wave1d_ah.u[alpha - 1 : -(alpha - 1) : alpha]
    diffs1d_znam = wave1d_ah.u[alpha - 1 : -(alpha - 1) : alpha] - wave1d_a2h.u[alpha**2 - alpha - 1 : -(alpha**2 - alpha - 1) : alpha ** 2]
    
    x = wave1d_h.t_i.size - 2
    # snap(n)
    
    norm1d_chisl_C = np.linalg.norm(diffs1d_chisl, ord= np.inf)
    norm1d_znam_C = np.linalg.norm(diffs1d_znam, ord= np.inf)
    beta1d_C[n] = norm1d_chisl_C / norm1d_znam_C

    norm1d_chisl_L2 = np.linalg.norm(diffs1d_chisl, ord= 2)
    norm1d_znam_L2 = np.linalg.norm(diffs1d_znam, ord= 2)
    beta1d_L2[n] = norm1d_chisl_L2 / norm1d_znam_L2
    
    n += 1

/var/folders/tx/t_y2r90n5gd84735fqzpxzcr0000gn/T/ipykernel_32026/3764389171.py:35: RuntimeWarning: divide by zero encountered in scalar divide
  beta1d_C[0] = norm1d_chisl_C / norm1d_znam_C
/var/folders/tx/t_y2r90n5gd84735fqzpxzcr0000gn/T/ipykernel_32026/3764389171.py:39: RuntimeWarning: divide by zero encountered in scalar divide
  beta1d_L2[0] = norm1d_chisl_L2 / norm1d_znam_L2


In [46]:
# image_path_list = [f'./img_test/img_{i}.png' for i in range(wave1d_h.t_i.size)]
# image_list = [Image.open(file) for file in image_path_list]
# image_list[0].save(
#         'test.gif',
#         save_all=True,
#         append_images=image_list[1:],
#         duration=0.01,
#         loop=0)

In [55]:
x = wave1d_h.t_i[:-1]

plt.figure(figsize=(10, 6), dpi= 250)
plt.plot(x, beta1d_C[:-1], label= 'C-norm')
plt.plot(x, beta1d_L2[:-1], label= 'L2-norm')
plt.plot(x, np.ones(x.size) * alpha**2, label= 'ref')
plt.legend()
plt.title(f'O2 grid convergence for 1D WE in R1', fontsize=14)
plt.grid(which='major', linestyle='-')
plt.grid(which='minor', linestyle='--')
plt.minorticks_on()
plt.xlabel('Time, s')
plt.savefig('./conv_1d.png', 
        transparent = False,  
        facecolor = 'white'
        )
plt.close()